In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_fomc_projections(url):
    """
    Fetches FOMC projection data from the given URL and returns two pandas DataFrames:
    one for central tendency and another for range projections.
    
    Parameters:
    url (str): The URL of the FOMC projections webpage.
    
    Returns:
    tuple: (central_tendency_df, range_df)
    """
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    
    if len(tables) < 2:
        raise Exception("Could not find the expected projection tables on the webpage.")
    
    def parse_table(table):
        rows = table.find_all('tr')
        data = []
        headers = [th.text.strip() for th in rows[0].find_all(['th', 'td'])]
        for row in rows[1:]:
            cols = [td.text.strip() for td in row.find_all(['th', 'td'])]
            if len(cols) == len(headers):  # Ensure correct column alignment
                data.append(cols)
        return pd.DataFrame(data, columns=headers) if data else pd.DataFrame()
    
    central_tendency_df = parse_table(tables[0])
    range_df = parse_table(tables[1])
    
    return central_tendency_df, range_df, tables

# Example usage
url = "https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20241218.htm"
central_tendency, range_df, tables = fetch_fomc_projections(url)
print(central_tendency)
print(range_df)





Empty DataFrame
Columns: []
Index: []
                                 2019  2020 2021 2022 2023 2024 2025 2026  \
0                         Actual  3.4  -1.0  5.7  1.3  3.2    -    -    -   
1             Upper End of Range    -     -    -    -    -  2.7  2.5  2.5   
2  Upper End of Central Tendency    -     -    -    -    -  2.5  2.2  2.1   
3                         Median    -     -    -    -    -  2.5  2.1  2.0   
4  Lower End of Central Tendency    -     -    -    -    -  2.4  1.8  1.9   
5             Lower End of Range    -     -    -    -    -  2.3  1.6  1.4   

  2027 Longer run  
0    -          -  
1  2.5        2.5  
2  2.0        2.0  
3  1.9        1.8  
4  1.8        1.7  
5  1.5        1.7  


In [4]:
tables

[<table aria-labelledby="xt1 xt1p1b" class="pubtables" data-sticky-columns="1" data-sticky-rows="2">
 <thead>
 <tr>
 <th class="colhead" id="xt1a1" rowspan="2">Variable</th>
 <th class="colhead" colspan="5" id="xt1a2">Median<a href="#xt1p1f1" id="xt1p1f1r"><sup>1</sup></a></th>
 <th class="colhead" colspan="5" id="xt1a3">Central Tendency<a href="#xt1p1f2" id="xt1p1f2r"><sup>2</sup></a></th>
 <th class="colhead" colspan="5" id="xt1a4">Range<a href="#xt1p1f3" id="xt1p1f3r"><sup>3</sup></a></th>
 </tr>
 <tr>
 <th class="colhead" headers="xt1a2" id="xt1b1">2024</th>
 <th class="colhead" headers="xt1a2" id="xt1b2">2025</th>
 <th class="colhead" headers="xt1a2" id="xt1b3">2026</th>
 <th class="colhead" headers="xt1a2" id="xt1b4">2027</th>
 <th class="colhead" headers="xt1a2" id="xt1b5">Longer run</th>
 <th class="colhead" headers="xt1a3" id="xt1b6">2024</th>
 <th class="colhead" headers="xt1a3" id="xt1b7">2025</th>
 <th class="colhead" headers="xt1a3" id="xt1b8">2026</th>
 <th class="colhead

In [5]:
def fetch_fomc_projection_links(calendar_url):
    """
    Extracts all FOMC projection links from the given FOMC calendar page.
    
    Parameters:
    calendar_url (str): The URL of the FOMC calendar webpage.
    
    Returns:
    list: A list of full URLs to FOMC projection materials.
    """
    base_url = "https://www.federalreserve.gov"
    response = requests.get(calendar_url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch calendar page. HTTP Status Code: {response.status_code}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    
    for a in soup.find_all('a', href=True):
        if 'projtabl' in a['href'] and a['href'].endswith('.htm'):
            links.append(base_url + a['href'])
    
    return links

# Example usage
calendar_url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
projection_links = fetch_fomc_projection_links(calendar_url)
print(projection_links)

['https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240320.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240612.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240918.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20241218.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230322.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230614.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230920.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20231213.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtable20220316.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20220615.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20220921.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20221214.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20210317.htm', 'https://www.federalreserve.gov/mone

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_fomc_projections(url):
    """
    Fetches all FOMC projection tables from the given URL and returns them as a dictionary of pandas DataFrames.
    
    Parameters:
    url (str): The URL of the FOMC projections webpage.
    
    Returns:
    dict: A dictionary where keys are table names and values are DataFrames.
    """
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    
    if not tables:
        raise Exception("No tables found on the webpage.")
    
    def parse_table(table):
        rows = table.find_all('tr')
        data = []
        headers = [th.text.strip() for th in rows[0].find_all(['th', 'td'])]
        for row in rows[1:]:
            cols = [td.text.strip() for td in row.find_all(['th', 'td'])]
            if len(cols) == len(headers):  # Ensure correct column alignment
                data.append(cols)
        return pd.DataFrame(data, columns=headers) if data else pd.DataFrame()
    
    tables_dict = {}
    for table in tables:
        caption = table.find_previous(['h4', 'h5'])
        table_name = caption.text.strip() if caption else f"Table_{len(tables_dict) + 1}"
        tables_dict[table_name] = parse_table(table)
    
    return tables_dict

def fetch_fomc_projection_links(calendar_url):
    """
    Extracts all FOMC projection links from the given FOMC calendar page.
    
    Parameters:
    calendar_url (str): The URL of the FOMC calendar webpage.
    
    Returns:
    list: A list of full URLs to FOMC projection materials.
    """
    base_url = "https://www.federalreserve.gov"
    response = requests.get(calendar_url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch calendar page. HTTP Status Code: {response.status_code}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    
    for a in soup.find_all('a', href=True):
        if 'projtabl' in a['href'] and a['href'].endswith('.htm'):
            links.append(base_url + a['href'])
    
    return links

# Example usage
calendar_url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
projection_links = fetch_fomc_projection_links(calendar_url)
print(projection_links)

# Example usage
url = "https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20241218.htm"
tables_dict = fetch_fomc_projections(url)
[(print(k), print(v)) for k, v in tables_dict.items()]


['https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240320.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240612.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240918.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20241218.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230322.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230614.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20230920.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20231213.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtable20220316.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20220615.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20220921.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20221214.htm', 'https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20210317.htm', 'https://www.federalreserve.gov/mone

[(None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None)]